In [ ]:
import numpy as np
from IPython.display import Image

NN_ARCHITECTURE = [
    {"input_dim": 4096, "output_dim": 512, "activation": "relu"},
    {"input_dim": 512, "output_dim": 512, "activation": "relu"},
    {"input_dim": 512, "output_dim": 4096, "activation": "sigmoid"},

]

def init_layers(nn_architecture, seed = 99):
# random seed initiation
    np.random.seed(seed)
# number of layers in our neural network
    number_of_layers = len(nn_architecture)
# parameters storage initiation
    params_values = {}

# iteration over network layers
    for idx, layer in enumerate(nn_architecture):
# we number network layers from 1
        layer_idx = idx + 1

# extracting the number of units in layers
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]

# initiating the values of the W matrix
# and vector b for subsequent layers
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1

    return params_values

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
# calculation of the input value for the activation function
    Z_curr = np.dot(W_curr, A_prev) + b_curr

# selection of activation function
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')

# return of calculated activation A and the intermediate Z matrix
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
# creating a temporary memory to store the information needed for a backward step
    memory = {}
# X vector is the activation for layer 0
    A_curr = X

# iteration over network layers
    for idx, layer in enumerate(nn_architecture):
# we number network layers from 1
        layer_idx = idx + 1
# transfer the activation from the previous iteration
        A_prev = A_curr

# extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]
# extraction of W for the current layer
        W_curr = params_values["W" + str(layer_idx)]
# extraction of b for the current layer
        b_curr = params_values["b" + str(layer_idx)]
# calculation of activation for the current layer
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)

# saving calculated values in the memory
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr

# return of prediction vector and a dictionary containing intermediate values
    return A_curr, memory

def get_cost_value(Y_hat, Y):  # Binary Crossentropy
# number of examples
    cost=0
    m = Y_hat.shape[1]
# print(m)
# print(Y_hat.shape)
# print(Y.shape)
# print(X.shape)
    for k in range(4096):
        cost1 = -1 / m * (np.dot(Y[k], np.log(Y_hat[k]).T) + np.dot(1 - Y[k], np.log(1 - Y_hat[k]).T))
# print(cost1)
        cost = cost+cost1

    cost = cost/4096
# calculation of the cost according to the formula
# cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

# NOT IN USE

def get_cost_value(Y_hat, Y):  # Mean square logarithmic error (MSLE)
# number of examples
    cost=0
    for k in range(4096):
# cost2 = -1 / m * ((np.dot((Y_hat[k]+1.),np.log(Y_hat[k]+1.).T) - np.dot((Y[k]+1.),np.log(Y[k]+1.).T))**2)
# cost2 = -1 / m * ((np.log(Y_hat[k]+1)) - (np.log(Y[k]+1)))**2
        cost2= np.mean(np.square(np.log(Y[k] + 1.) - np.log(Y_hat[k] + 1.)), axis=-1)

# print(Y_hat[k])
        cost = cost+cost2
# calculation of the cost according to the formula
# cost = -1 / m * ((np.log(Y_hat+1).T) + (np.log(1 + Y).T))**2
    cost = cost/4096
    return np.squeeze(cost)

# an auxiliary function that converts probability into class
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
# number of examples
    m = A_prev.shape[1]

# selection of activation function
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')

# calculation of the activation function derivative
    dZ_curr = backward_activation_func(dA_curr, Z_curr)

# derivative of the matrix W
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
# derivative of the vector b
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
# derivative of the matrix A_prev
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}

# print(Y.shape)
# number of examples
    m = Y.shape[1]
# print(m)
# a hack ensuring the same shape of the prediction vector and labels vector
    Y = Y.reshape(Y_hat.shape)

# initiation of gradient descent algorithm
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));

    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
# we number network layers from 1
        layer_idx_curr = layer_idx_prev + 1
# extraction of the activation function for the current layer
        activ_function_curr = layer["activation"]

        dA_curr = dA_prev

        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]

        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]

        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)

        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr

    return grads_values

def update(params_values, grads_values, nn_architecture, learning_rate):

# iteration over network layers
    for layer_idx, layer in enumerate(nn_architecture, 1):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values;

# ## Traditional Training

# def train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):

def train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):
# initiation of neural net parameters
    params_values = init_layers(nn_architecture, 2)
# ##print(params_values)
# params_values=out
# print(params_values[0].shape)

# initiation of lists storing the history
# of metrics calculated during the learning process
    cost_history = []
    accuracy_history = []

# performing calculations for subsequent iterations
    for i in range(epochs):
# model.train()
# step forward
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)

# calculating metrics and saving them in history
        cost = get_cost_value(Y_hat, Y)
# cost_history.append(cost)
# accuracy = get_accuracy_value(Y_hat, Y)
# accuracy_history.append(accuracy)

# step backward - calculating gradient
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
# print(grads_values)
# updating model state
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
# ##print(params_values)
# if (i > epochs):
# params_values = train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None, params_values1=params_values)

# if(i % 50 == 0):
        if(i % 1 == 0):
            if(verbose):
# print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, float(cost), float(accuracy)))
# print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, cost, accuracy))
# ###print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, cost, accuracy))
                print("Iteration: {:05} - cost: {:.5f}".format(i, cost))
# # print("Iteration: {:05}".format(i))
# # print("cost: {:.5f}".format,cost)
# print("cost: {:.5f}".format(cost))
# # print("accuracy: {:.5f}".format,accuracy)
# list(map('{:.5f}%'.format,cost))
# print('{},{}'.format(cost[:.5], accuracy[:.5]))

            if(callback is not None):
                callback(i, params_values)
# output=params_values

# for i in range(epochs):
# return train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None, out=output)
    return params_values
# out=params_values
# return train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None, params_values)
# params_values1 = params_values
###############################################################
# ## Online traning
###############################################################
# params_values1=0
def train(sample, X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):
    global params_values
# initiation of neural net parameters
# for s in range(sample):
    s=sample
    if (s == 0):
# print(s)
        params_values = init_layers(nn_architecture, 2)
    else:
# if (s > 0):
        print(s)
# params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        params_values = params_values
# params_values1 = params_values

# print(params_values1)

# ##print("Initial param value")
    params_values = params_values

# ##print(params_values)
# print("Initial param value: {:05}".format,params_values)

# if (ee < sample):
# params_values = params_values
# initiation of lists storing the history
# of metrics calculated during the learning process
    cost_history = []
    accuracy_history = []

# performing calculations for subsequent iterations
    for i in range(epochs):
# step forward
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)

# calculating metrics and saving them in history
        cost = get_cost_value(Y_hat, Y)
# cost_history.append(cost)
# accuracy = get_accuracy_value(Y_hat, Y)
# accuracy_history.append(accuracy)

# step backward - calculating gradient
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
# print(grads_values)
# updating model state
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
# #params_values_prev = params_values
# #params_values_curr= params_values_prev[[epochs]]
# print("updated param value: {:05}".format,params_values)
# ##print("updated param value")
# ##print(params_values)
# #if (i > epochs):
# params_values = train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None, params_values1=params_values)
# #    params_values_curr = train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None)
# if(i % 50 == 0):
        if(i % 1 == 0):
            if(verbose):
# print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, float(cost), float(accuracy)))
                print("Iteration: {:05} - cost: {:.5f}".format(i, cost))
# plt.plot(cost)
            if(callback is not None):
                callback(i, params_values)
# #output=params_values
# return train(X, Y, nn_architecture, epochs-1, learning_rate, verbose=False, callback=None)
# return train()
# ee=ee+1
# else:
# params_values=params_values
# train(sample,X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None)
# global params_values
    return params_values

# return train(output,X, Y, nn_architecture, epochs-1, learning_rate, verbose=False, callback=None)
# params_values = params_values

import os
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("whitegrid")

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras import regularizers

from sklearn.metrics import accuracy_score

# number of samples in the data set
N_SAMPLES = 1560
# ratio between training and test sets
TEST_SIZE = 0.1

import scipy.io as spio
mat = spio.loadmat('<Folder Path>\<File Name>.mat', squeeze_me=True)
# inputt=mat.get('Covariance')
# grth=mat.get('Precision')
# inputt=mat.get('res_Cov_Final')
# #X=mat.get('res_Cov_Final')
X=mat.get('res_Cov_Final_nor')
X=np.array(X)
y=mat.get('Laplacian_nor')
y=np.array(y)
# print(inputt)
# print(inputt.shape)
# print(grth)
# print(grth.shape)
print(X)
print(X.shape)
print(y)
print(y.shape)

# y.shape[1]
# Training for Traditional Training
# params_values = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10000, 0.01)[0]
params_values = train(np.transpose(X), np.transpose(y), NN_ARCHITECTURE, 10, 0.0001,verbose=1)
# plt.plot(train['cost'])
# ###params_values = train(np.transpose(X[1]), np.transpose(y[1]), NN_ARCHITECTURE, 10, 0.0001,verbose=1)
# params_values = train(np.transpose(X.flatten()), np.transpose(y.flatten()), NN_ARCHITECTURE, 10000, 0.01,verbose=1)
# print(params_values)

# ### Training for Online traning

# print(X[0][0])
# print([X[0]])
# a=np.array([X[0]])
# print(a.shape)
# ya=np.array([y[0]])
# yt=ya.T
# print(yt)
# print(ya.shape[0])
# print(yt.shape)
# print(y[1])
# b=(np.transpose(X[0]))
# print(b)
# c=((X[0].T))
# print(c)
# print(b.shape)
# print(c.shape)
# print(y[1].shape[1])
# Training
# params_values = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10000, 0.01)[0]
# ######params_values = train(np.transpose(X), np.transpose(y), NN_ARCHITECTURE, 10, 0.0001,verbose=1)
# params_values = train(np.transpose(X.flatten()), np.transpose(y.flatten()), NN_ARCHITECTURE, 10000, 0.01,verbose=1)
# print(params_values)
for t in range(1560):
# params_values1 = params_values
# print(params_values1)
    print("Sample: {:04}".format(t))
# params_values = train(np.transpose([X[t]]), np.transpose([y[t]]), NN_ARCHITECTURE, 5, 0.0001,verbose=1)
# print(t)

# ## Traditional Training
# params_values = train(np.transpose([X[t]]), np.transpose([y[t]]), NN_ARCHITECTURE, 10, 0.0001,verbose=1)

# ## Online traning
    params_values = train(t, np.transpose([X[t]]), np.transpose([y[t]]), NN_ARCHITECTURE, 10, 0.0001,verbose=1)
# aaa=train.params_values
# print(params_values)
# print("Sample: {:04}".format(t))
# params_values = train(np.transpose([X[t]]), np.transpose([y[t]]), params_values, 10, 0.0001,verbose=1)
# return params_values

# print(params_values1)

train(0, np.transpose([X[t]]), np.transpose([y[t]]), NN_ARCHITECTURE, 10, 0.0001,verbose=1)

Y_test_hat, _ = full_forward_propagation(np.transpose(X), params_values, NN_ARCHITECTURE)

print(Y_test_hat)

np.savetxt('Predicted_data.mat', Y_test_hat)
